In [1]:
import random
import numpy as np
import time
import numba
import math
from numpy.random import Generator, PCG64DXSM, SeedSequence
import multiprocessing as mp
from structure_ORIGINAL_FNs_00 import play_sequence

np.set_printoptions(suppress=True)

In [2]:
# use this cell for setting initial perameters

# rng = np.random.default_rng()

sides = 1
pairs = np.asarray([[0, 1], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3]])
testpairs = np.asarray([[1, 3], [3, 1]])
nonadjpairs = np.asarray([[0, 2], [2, 0], [0, 3], [3, 0], [0, 4], [4, 0], [1, 4], [4, 1], [2, 4], [4, 2]])
allpairs = np.concatenate((pairs, nonadjpairs, testpairs))

plen = len(pairs)
alen = len(allpairs)

terms = 5
# maxvalue = 5
startstop = 2
# noise = 0.2
annealing = 0.00001

timesteps = 3*10**7
runs = 100

# rein1 = 4
# rein2 = 0
# punish1 = 0
# punish2 = -11
inertia = 1

nsteps = 100
# blocklength = nsteps*10


In [3]:
# use this cell for running the code

#measuring how long code takes to run
start = time.perf_counter()



#nprocs = mp.cpu_count()-1
pool = mp.Pool(processes=20)

sg = SeedSequence()
rgs = [Generator(PCG64DXSM(s)) for s in sg.spawn(runs)]

trainrates = []
testrates = []

for blocklength in [1000, 10000]:
    for maxvalue in [5, 10, 15, 20]:
        for noise in [0., .2]:
            for rein1 in range(2, 7, 2):
                rein2 = rein1 
                for punish1 in range(0, -19, -3):
                    punish2 = punish1
                    
                    
                    mpseq_final = pool.starmap(play_sequence, [(n, rgs[n], rein1, punish1, rein2, punish2, timesteps, nsteps, sides, pairs, testpairs, nonadjpairs, allpairs, plen, alen, terms, maxvalue, startstop, noise, annealing, runs, inertia, blocklength) for n in range(runs)])

                    final_sigweights = []
                    final_cumsuc = []
                    final_cumsucadd = []
                    final_testcumsucadd = []
                    final_recweights = []

                    for res_idx in range(0, len(mpseq_final)):

                        final_sigweights.append(mpseq_final[res_idx][0])
                        final_cumsuc.append(mpseq_final[res_idx][1])
                        final_cumsucadd.append(mpseq_final[res_idx][2])
                        final_testcumsucadd.append(mpseq_final[res_idx][3])
                        final_recweights.append(mpseq_final[res_idx][4])

                    final_sigweights = np.asarray(final_sigweights)
                    final_cumsuc = np.asarray(final_cumsuc)
                    final_cumsucadd = np.asarray(final_cumsucadd)
                    final_testcumsucadd = np.asarray(final_testcumsucadd)
                    final_recweights = np.asarray(final_recweights)



                    xtime = time.localtime()
                    current_time = time.strftime("%H:%M:%S", xtime)
                    print(f"time = {current_time}")
                    print("average cumsuc = ")
                    print(np.average(final_cumsuc)/runs)

                    print("average final cumsucadd = ")
                    print(np.average(final_cumsucadd)/(100))
                    trainrates.append(np.average(final_cumsucadd)/(100))

                    print("average test cumsum = ")
                    print(np.average(final_testcumsucadd)/(100))
                    testrates.append(np.average(final_testcumsucadd)/(100))
                    print(f'rein1 = {rein1} rein2 = {rein2} punish1 = {punish1} punish2 = {punish2}')
                    print(f'blocklengh = {blocklength} maxvalue = {maxvalue} noise = {noise}')
                    print(" ")
                    print(" ")


finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

time = 10:22:31
average cumsuc = 
269147.3271
average final cumsucadd = 
0.9143000000000001
average test cumsum = 
0.5442
rein1 = 2 rein2 = 2 punish1 = 0 punish2 = 0
blocklengh = 1000 maxvalue = 5 noise = 0.0
 
 
time = 10:27:28
average cumsuc = 
296828.0586
average final cumsucadd = 
0.9904999999999999
average test cumsum = 
0.6620999999999999
rein1 = 2 rein2 = 2 punish1 = -3 punish2 = -3
blocklengh = 1000 maxvalue = 5 noise = 0.0
 
 
time = 10:32:27
average cumsuc = 
299817.0173
average final cumsucadd = 
1.0
average test cumsum = 
0.7662
rein1 = 2 rein2 = 2 punish1 = -6 punish2 = -6
blocklengh = 1000 maxvalue = 5 noise = 0.0
 
 
time = 10:37:26
average cumsuc = 
293901.8749
average final cumsucadd = 
0.9887
average test cumsum = 
0.7454999999999999
rein1 = 2 rein2 = 2 punish1 = -9 punish2 = -9
blocklengh = 1000 maxvalue = 5 noise = 0.0
 
 
time = 10:42:26
average cumsuc = 
276370.1983
average final cumsucadd = 
0.9529000000000001
average test cumsum = 
0.8069
rein1 = 2 rein2 = 2 pun

time = 13:48:16
average cumsuc = 
227482.8272
average final cumsucadd = 
0.9962000000000001
average test cumsum = 
0.7089
rein1 = 6 rein2 = 6 punish1 = -15 punish2 = -15
blocklengh = 1000 maxvalue = 5 noise = 0.2
 
 
time = 13:53:32
average cumsuc = 
223619.95260000002
average final cumsucadd = 
0.9973000000000001
average test cumsum = 
0.7040000000000001
rein1 = 6 rein2 = 6 punish1 = -18 punish2 = -18
blocklengh = 1000 maxvalue = 5 noise = 0.2
 
 
time = 14:00:01
average cumsuc = 
253684.65170000002
average final cumsucadd = 
0.8595999999999999
average test cumsum = 
0.5313
rein1 = 2 rein2 = 2 punish1 = 0 punish2 = 0
blocklengh = 1000 maxvalue = 10 noise = 0.0
 
 
time = 14:06:04
average cumsuc = 
297797.4341
average final cumsucadd = 
0.9941
average test cumsum = 
0.6085
rein1 = 2 rein2 = 2 punish1 = -3 punish2 = -3
blocklengh = 1000 maxvalue = 10 noise = 0.0
 
 
time = 14:12:11
average cumsuc = 
283583.1872
average final cumsucadd = 
0.9664
average test cumsum = 
0.7152
rein1 = 2 re

time = 17:54:18
average cumsuc = 
164246.4174
average final cumsucadd = 
0.5548
average test cumsum = 
0.5027
rein1 = 6 rein2 = 6 punish1 = -3 punish2 = -3
blocklengh = 1000 maxvalue = 10 noise = 0.2
 
 
time = 18:01:05
average cumsuc = 
187770.29390000002
average final cumsucadd = 
0.6933
average test cumsum = 
0.49310000000000004
rein1 = 6 rein2 = 6 punish1 = -6 punish2 = -6
blocklengh = 1000 maxvalue = 10 noise = 0.2
 
 
time = 18:07:36
average cumsuc = 
213840.3323
average final cumsucadd = 
0.8909
average test cumsum = 
0.5977
rein1 = 6 rein2 = 6 punish1 = -9 punish2 = -9
blocklengh = 1000 maxvalue = 10 noise = 0.2
 
 
time = 18:14:12
average cumsuc = 
219903.49649999998
average final cumsucadd = 
0.9788
average test cumsum = 
0.6607999999999999
rein1 = 6 rein2 = 6 punish1 = -12 punish2 = -12
blocklengh = 1000 maxvalue = 10 noise = 0.2
 
 
time = 18:20:50
average cumsuc = 
215822.3403
average final cumsucadd = 
0.9964
average test cumsum = 
0.6693000000000001
rein1 = 6 rein2 = 6 p

time = 22:51:43
average cumsuc = 
180964.3081
average final cumsucadd = 
0.8686
average test cumsum = 
0.7424
rein1 = 4 rein2 = 4 punish1 = -15 punish2 = -15
blocklengh = 1000 maxvalue = 15 noise = 0.2
 
 
time = 23:00:03
average cumsuc = 
169116.0433
average final cumsucadd = 
0.7202
average test cumsum = 
0.7269
rein1 = 4 rein2 = 4 punish1 = -18 punish2 = -18
blocklengh = 1000 maxvalue = 15 noise = 0.2
 
 
time = 23:08:27
average cumsuc = 
157201.4732
average final cumsucadd = 
0.5292
average test cumsum = 
0.5028
rein1 = 6 rein2 = 6 punish1 = 0 punish2 = 0
blocklengh = 1000 maxvalue = 15 noise = 0.2
 
 
time = 23:16:46
average cumsuc = 
161043.8439
average final cumsucadd = 
0.5524
average test cumsum = 
0.5045000000000001
rein1 = 6 rein2 = 6 punish1 = -3 punish2 = -3
blocklengh = 1000 maxvalue = 15 noise = 0.2
 
 
time = 23:24:46
average cumsuc = 
187496.7507
average final cumsucadd = 
0.6856
average test cumsum = 
0.517
rein1 = 6 rein2 = 6 punish1 = -6 punish2 = -6
blocklengh = 10

time = 04:32:52
average cumsuc = 
201368.93010000003
average final cumsucadd = 
0.8584
average test cumsum = 
0.5885
rein1 = 4 rein2 = 4 punish1 = -6 punish2 = -6
blocklengh = 1000 maxvalue = 20 noise = 0.2
 
 
time = 04:42:03
average cumsuc = 
200220.965
average final cumsucadd = 
0.985
average test cumsum = 
0.6555
rein1 = 4 rein2 = 4 punish1 = -9 punish2 = -9
blocklengh = 1000 maxvalue = 20 noise = 0.2
 
 
time = 04:51:31
average cumsuc = 
178595.2355
average final cumsucadd = 
0.8315
average test cumsum = 
0.695
rein1 = 4 rein2 = 4 punish1 = -12 punish2 = -12
blocklengh = 1000 maxvalue = 20 noise = 0.2
 
 
time = 05:01:11
average cumsuc = 
159517.3945
average final cumsucadd = 
0.6181
average test cumsum = 
0.6406999999999999
rein1 = 4 rein2 = 4 punish1 = -15 punish2 = -15
blocklengh = 1000 maxvalue = 20 noise = 0.2
 
 
time = 05:10:45
average cumsuc = 
154740.6563
average final cumsucadd = 
0.5589
average test cumsum = 
0.578
rein1 = 4 rein2 = 4 punish1 = -18 punish2 = -18
blockle

...

Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-16:
Process ForkPoolWorker-1:
Process ForkPoolWorker-9:
Process ForkPoolWorker-15:
Process ForkPoolWorker-2:
Process ForkPoolWorker-17:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-20:
Process ForkPoolWorker-5:
Process ForkPoolWorker-18:
Process ForkPoolWorker-11:
Process ForkPoolWorker-19:
Process ForkPoolWorker-10:
Process ForkPoolWorker-13:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/h

  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/nothin/python/structure2/structure_ORIGINAL_FNs_00.py", line 243, in play_sequence
    recweights, sigweights, cumsucadd = nstepsfn(noise, plen, alen, maxvalue, naturesides, recweights, sigweights, randunif2, randunif4, randunif6, randunif8, randunif10, naturestates, nsteps, allpairs, rein, punish)
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/nothin/miniconda3/lib/python3.9/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0],

KeyboardInterrupt
  File "/home/nothin/python/structure2/structure_ORIGINAL_FNs_00.py", line 175, in randoms
    fnaturestates1 = frng.integers(fplen1*falen1, size=fnsteps1)
  File "/home/nothin/python/structure2/structure_ORIGINAL_FNs_00.py", line 243, in play_sequence
    recweights, sigweights, cumsucadd = nstepsfn(noise, plen, alen, maxvalue, naturesides, recweights, sigweights, randunif2, randunif4, randunif6, randunif8, randunif10, naturestates, nsteps, allpairs, rein, punish)
KeyboardInterrupt
  File "/home/nothin/python/structure2/structure_ORIGINAL_FNs_00.py", line 243, in play_sequence
    recweights, sigweights, cumsucadd = nstepsfn(noise, plen, alen, maxvalue, naturesides, recweights, sigweights, randunif2, randunif4, randunif6, randunif8, randunif10, naturestates, nsteps, allpairs, rein, punish)
KeyboardInterrupt
  File "/home/nothin/python/structure2/structure_ORIGINAL_FNs_00.py", line 243, in play_sequence
    recweights, sigweights, cumsucadd = nstepsfn(noise, plen, ale

In [4]:
trainrates = np.asarray(trainrates)
testrates = np.asarray(testrates)
np.save('structure_noiseAnn_dsktp_ORIGINAL_ONEside-SPREADc1_trainrates', trainrates)
np.save('structure_noiseAnn_dsktp_ORIGINAL_ONEside-SPREADc1_testrates', testrates)



In [4]:
idxy = 2
sigweights = final_sigweights[idxy]
cumsucadd = final_cumsucadd[idxy]
testcumsucadd = final_testcumsucadd[idxy]
recweights = final_recweights[idxy]

print(cumsucadd)
print(testcumsucadd)

91.0
92.0


In [5]:

simple_sig = np.zeros_like(sigweights)
for ax0 in range(0, sides):
    for ax1 in range(0, 2):
        for ax2 in range(0, terms):
            for ax3 in range(0, maxvalue):
                if (sigweights[ax0, ax1, ax2, ax3][0]) > (sigweights[ax0, ax1, ax2, ax3][1]):
                    simple_sig[ax0, ax1, ax2, ax3][0] = 1
                else:
                    simple_sig[ax0, ax1, ax2, ax3][1] = 1

simple_sig = np.sum(simple_sig, axis=3)
print(simple_sig)

for s in range(0, sides):
    for l in range(0, terms):
        for s2 in range(0, sides):
            for r in range(0, terms):
                if l != r:
                    print(f'LEFT side: {s} term: {l} magnitude: {simple_sig[s, 0, l, 1]}')
                    print(f'RIGHT side: {s2} term: {r} magnitude: {simple_sig[s2, 1, r, 1]}')
                    print(f'receiver: {recweights[math.floor(simple_sig[s, 0, l, 1]*(maxvalue+1) + simple_sig[s2, 1, r, 1])]}')

[[[[0. 5.]
   [4. 1.]
   [1. 4.]
   [1. 4.]
   [3. 2.]]

  [[3. 2.]
   [3. 2.]
   [2. 3.]
   [0. 5.]
   [0. 5.]]]]
LEFT side: 0 term: 0 magnitude: 5.0
RIGHT side: 0 term: 1 magnitude: 2.0
receiver: [     1. 203123.]
LEFT side: 0 term: 0 magnitude: 5.0
RIGHT side: 0 term: 2 magnitude: 3.0
receiver: [     1. 221225.]
LEFT side: 0 term: 0 magnitude: 5.0
RIGHT side: 0 term: 3 magnitude: 5.0
receiver: [63918.     1.]
LEFT side: 0 term: 0 magnitude: 5.0
RIGHT side: 0 term: 4 magnitude: 5.0
receiver: [63918.     1.]
LEFT side: 0 term: 1 magnitude: 1.0
RIGHT side: 0 term: 0 magnitude: 2.0
receiver: [2573823.       1.]
LEFT side: 0 term: 1 magnitude: 1.0
RIGHT side: 0 term: 2 magnitude: 3.0
receiver: [4627151.       1.]
LEFT side: 0 term: 1 magnitude: 1.0
RIGHT side: 0 term: 3 magnitude: 5.0
receiver: [    1. 92934.]
LEFT side: 0 term: 1 magnitude: 1.0
RIGHT side: 0 term: 4 magnitude: 5.0
receiver: [    1. 92934.]
LEFT side: 0 term: 2 magnitude: 4.0
RIGHT side: 0 term: 0 magnitude: 2.0
receiver

In [13]:
np.set_printoptions(threshold=np.inf)
print(sigweights)

[[[[[ 6964559.        1.]
    [ 6964475.        1.]
    [ 6964503.        1.]
    [ 6964449.        1.]
    [ 6964449.        1.]
    [ 6964523.        1.]
    [       1.  6964523.]
    [ 6964525.        1.]
    [       1.  6964489.]
    [       1.  6964457.]
    [       1.  6964529.]
    [ 6964517.        1.]
    [ 6964489.        1.]
    [       1.  6964469.]
    [ 6964475.        1.]
    [ 6964483.        1.]
    [ 6964457.        1.]
    [       1.  6964465.]
    [       1.  6964485.]
    [       1.  6964469.]]

   [[       7. 12661503.]
    [ 7302905.  5358561.]
    [      13. 12661457.]
    [12661483.        1.]
    [       1. 12661461.]
    [      19. 12661453.]
    [       1. 12661465.]
    [      49. 12661383.]
    [      11. 12661421.]
    [12661461.       13.]
    [      25. 12661439.]
    [12661467.        1.]
    [       1. 12661473.]
    [12661527.        1.]
    [       7. 12661497.]
    [       1. 12661473.]
    [12661499.        1.]
    [12661515.        7.]
    [12661